In [38]:
import kipoi
from kipoi_veff import score_variants
import pandas as pd
from kipoi_cadd.utils import load_pickle, decompose_variant_string, generate_intervals_file
from kipoi_cadd.writers import LmdbWriter
import kipoi_veff.snv_predict as sp
from kipoi_veff.scores import Diff, LogitRef, Logit
from kipoi import get_model

## Get DeepSea features

In [4]:
training_dir = "/s/project/kipoi-cadd/data/raw/v1.3/training_data/"
sample_intervals_file = "/data/nasif12/home_if12/simancas/.kipoi/models/DeepSEA/variantEffects/downloaded/example_files/intervals_file"
sample_fasta_file = "/data/nasif12/home_if12/simancas/.kipoi/models/DeepSEA/variantEffects/downloaded/example_files/fasta_file"
sample_vcf_file = "/data/nasif12/home_if12/simancas/.kipoi/models/DeepSEA/variantEffects/downloaded/example_files/variants.vcf"
test_dir = "/tmp/kipoi-veff/test_KipoiLmdbDataset/"
kipoi_features_dir = "/s/project/kipoi-cadd/data/processed/v1.3/kipoi_features/"
intervals_file = kipoi_features_dir + "intervals_10k.tsv"
fasta_file = "/s/genomes/human/hg19/ensembl_GRCh37.p13_release75/Homo_sapiens.GRCh37.75.dna.primary_assembly.fa"
shuff_10k_file = training_dir + "ids_10k.pkl"

In [3]:
# To trigger the download of sample intervals and sample fasta files, we have to do this:
model = get_model('DeepSEA/variantEffects')
pred = model.pipeline.predict_example()

Using downloaded and verified file: /data/nasif12/home_if12/simancas/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1


100%|██████████| 1/1 [00:00<00:00,  2.96it/s]


In [13]:
sample_intervals = pd.read_csv(sample_intervals_file, sep='\t', header=None, names=['chr', 'start', 'end'])
"""
with open(sample_fasta_file, "r") as f:
    for line in f:
        print(line)
        break
"""
sample_intervals.head()

,chr,start,end
0,chr22,136018,136069
1,chr22,136351,136402
2,chr22,137749,137800
3,chr22,134503,134554
4,chr22,139010,139061


## Generate intervals file for the variants of our interest

In [32]:
var_ids = load_pickle(shuff_10k_file)
generate_intervals_file(var_ids, intervals_file, use_chr_word=False, header=None)

In [ ]:
model = get_model('DeepSEA/variantEffects')
dl_kwargs = {'intervals_file': intervals_file, 'fasta_file': fasta_file}

In [7]:
# Get the dataloader and instantiate it
dl = model.default_dataloader(**{'intervals_file': intervals_file, 'fasta_file': fasta_file, 'num_chr_fasta': True})
it = dl.batch_iter(batch_size=64)
preds = None
for batch in it:
    if preds is None:
        preds = model.predict_on_batch(batch['inputs'])
    else:
        preds = np.concatenate([preds, model.predict_on_batch(batch['inputs'])], axis=0)
preds.shape

array([[3.98645550e-02, 5.88591094e-04, 8.94931033e-02, ...,
        6.55408902e-03, 6.56132549e-02, 7.31519517e-03],
       [1.02449907e-03, 2.00078313e-04, 1.07573438e-03, ...,
        2.59452909e-02, 7.92154744e-02, 1.52810579e-04],
       [6.32132776e-03, 1.06681315e-02, 7.64664495e-03, ...,
        7.73444101e-02, 3.92744362e-01, 1.42737338e-02],
       ...,
       [1.20406123e-02, 1.59813259e-02, 1.28443195e-02, ...,
        4.10337657e-01, 5.05032957e-01, 1.27899591e-02],
       [4.51430771e-03, 3.04261479e-03, 5.15099289e-03, ...,
        7.72295427e-03, 8.25386643e-02, 5.23434347e-03],
       [3.54196847e-04, 3.60575541e-05, 2.24278177e-04, ...,
        6.14551734e-03, 4.21312340e-02, 1.65928795e-04]], dtype=float32)

## Write results with LMDBWriter for other Kipoi predictions

In [5]:
# Standalone script
import kipoi
from kipoi_cadd.writers import LmdbWriter
import kipoi_veff.snv_predict as sp
from kipoi_veff.scores import Logit
from kipoi import get_model

model_dir = "/data/nasif12/home_if12/simancas/.kipoi/models/DeepSEA/variantEffects/"
test_dir = "/tmp/kipoi-veff/test_KipoiLmdbDataset/"

sample_intervals_file = model_dir + "downloaded/example_files/intervals_file"
sample_fasta_file = model_dir + "downloaded/example_files/fasta_file"
sample_vcf_file = model_dir + "downloaded/example_files/variants.vcf"
lmdb_deep_sea = test_dir + "lmdb_DeepSea"

model = get_model(model_dir, source="dir")
dl_kwargs = {'intervals_file': sample_intervals_file, 'fasta_file': sample_fasta_file, 'num_chr_fasta': True}
dataloader = model.default_dataloader(**dl_kwargs).seq_dl
writer = LmdbWriter(lmdb_deep_sea)

sp.predict_snvs(model,
                dataloader,
                sample_vcf_file,
                64,
                num_workers=1,
                dataloader_args=dl_kwargs,
                evaluation_function_kwargs={'diff_types': {'logit': Logit()}},
                sync_pred_writer=writer)

Using downloaded and verified file: /data/nasif12/home_if12/simancas/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1


Exception: String output definition not recognized in array string converter!

In [47]:
from kipoi_veff.utils import ModelInfoExtractor
from kipoi_veff.utils.generic import _get_seq_shape_model, _get_seq_fields
from kipoi_veff.utils.dna_reshapers import ReshapeDnaString
ModelInfoExtractor(model, dataloader)

Exception: String output definition not recognized in array string converter!

In [51]:
ReshapeDnaString(str(_get_seq_shape_model(model, _get_seq_fields(model))))

Exception: String output definition not recognized in array string converter!

In [53]:
len(_get_seq_shape_model(model, _get_seq_fields(model)))

3

In [56]:
_get_seq_shape_model(model, _get_seq_fields(model))

(4, 1, 1000)

In [55]:
model.__dir__()

['model',
 'use_cuda',
 'grad_hooks',
 'type',
 'args',
 'info',
 'schema',
 'dependencies',
 'test',
 'default_dataloader',
 'name',
 'source',
 'source_name',
 'source_dir',
 'postprocessing',
 'pipeline',
 '__module__',
 '__doc__',
 'MODEL_PACKAGE',
 '__init__',
 'correct_neg_stride',
 '_torch_var',
 '_torch_var_to_numpy',
 '_model_is_cuda',
 'predict_on_batch',
 'numpy_to_torch',
 'np_run_pred',
 'pred_to_np',
 'get_layer',
 'get_layer_id',
 'extract_module_id',
 '_is_nonlinear_activation',
 '_get_trace',
 'get_last_layers',
 'get_downstream_layers',
 '_pt_type_match',
 'get_grad_tens',
 '_register_fwd_hook',
 '_input_grad',
 'input_grad',
 'get_upstream_layers',
 'predict_activation_on_batch',
 '__metaclass__',
 '_sufficient_deps',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 

In [ ]:
*